# DATA COLLECTION
we used the python beautifulsoup library to scrape data off sgcarmart.com to retrieve the information that we needed  

In [29]:
import sys
sys.path.append("..\src")

In [30]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import time
from datetime import datetime

from sgcarmart import *

In [31]:
from sgcarmart import power_retrieval

We aimed to use 4000 different listings to gather our data

In [32]:
main_page_listing_list = []
for idx, link in enumerate(range(40)):
    url = "https://www.sgcarmart.com/used_cars/listing.php?BRSR=" + str(idx * 100) + "&RPG=100&AVL=2&VEH=2"
    main_page_listing_list.append(url)

In [33]:
print(main_page_listing_list,'\n','\n', len(main_page_listing_list))

['https://www.sgcarmart.com/used_cars/listing.php?BRSR=0&RPG=100&AVL=2&VEH=2', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=100&RPG=100&AVL=2&VEH=2', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=200&RPG=100&AVL=2&VEH=2', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=300&RPG=100&AVL=2&VEH=2', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=400&RPG=100&AVL=2&VEH=2', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=500&RPG=100&AVL=2&VEH=2', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=600&RPG=100&AVL=2&VEH=2', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=700&RPG=100&AVL=2&VEH=2', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=800&RPG=100&AVL=2&VEH=2', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=900&RPG=100&AVL=2&VEH=2', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=1000&RPG=100&AVL=2&VEH=2', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=1100&RPG=100&AVL=2&VEH=2', 'https://www.sgcarmart.com/used_cars/li

Here we use the base link to retrieve the individual listings of cars. We then store all the links of individual listings and remove any duplicate listing urls

In [34]:
# Base url, or you can think of this as the individual car listing prefix
base_url = 'https://www.sgcarmart.com/used_cars/'
listing_urls = []

# Acquiring indvidual car listings    
for main_link in main_page_listing_list:
   
    # Make a request to the website and get the object
    content = requests.get(main_link)

    # Parse the HTML text
    soup = BeautifulSoup(content.text, 'lxml')

    # This returns a list of every tag that contains a link in one main link (each element in main page listing)
    links = soup.find_all('a')
    
    # Create a list for storing all the individual listing urls
    
    for link in links:
        # Get link in <a href>
        suffix = link.get('href')

        # Check if 'ID=' and 'DL=' exist in the string
        if ('ID=' in suffix) and ('DL=' in suffix):

            # Concatenate the two strings if they do
            listing_url = base_url + suffix
            
            # Append result to the list
            listing_urls.append(listing_url)
            
#     Removing duplicates
    set_listing_urls = set(listing_urls)
    listing_urls = list(set_listing_urls)
    
    # Prevent oneself from getting blocked from the website
    time.sleep(1)

Here we check the amount of listings we have after removing the duplicates.
We had 4000 to begin with and 3985 after removing duplicate urls

In [35]:
print(len(listing_urls))
print(len(set(listing_urls)))
print(len(list(set(listing_urls))))

3999
3999
3999


In [36]:
print(listing_urls[:10])

['https://www.sgcarmart.com/used_cars/info.php?ID=1283084&DL=1000&utm_content=SLeligible', 'https://www.sgcarmart.com/used_cars/info.php?ID=1289791&DL=4157', 'https://www.sgcarmart.com/used_cars/info.php?ID=1283706&DL=3723', 'https://www.sgcarmart.com/used_cars/info.php?ID=1280512&DL=3626&utm_content=SLeligible', 'https://www.sgcarmart.com/used_cars/info.php?ID=1213190&DL=2493&utm_content=SLeligible', 'https://www.sgcarmart.com/used_cars/info.php?ID=1288812&DL=3630', 'https://www.sgcarmart.com/used_cars/info.php?ID=1278869&DL=3648&utm_content=SLeligible', 'https://www.sgcarmart.com/used_cars/info.php?ID=1289114&DL=4108', 'https://www.sgcarmart.com/used_cars/info.php?ID=1282535&DL=3214&utm_content=SLeligible', 'https://www.sgcarmart.com/used_cars/info.php?ID=1253721&DL=3968&utm_content=SLeligible']


We create our dataframe and populate the columns with information that we need

In [37]:
df = pd.DataFrame(columns=['LISTING_URL', 'BRAND', 'PRICE', 'DEPRE_VALUE_PER_YEAR',
       'REG_DATE', 'MILEAGE_KM', 'MANUFACTURED_YEAR',
       'ROAD_TAX_PER_YEAR','TRANSMISSION', 'DEREG_VALUE_FROM_SCRAPE_DATE',
       'SCRAPE_DATE', 'OMV', 'ARF','POWER', 'COE_FROM_SCRAPE_DATE',
       'DAYS_OF_COE_LEFT', 'ENGINE_CAPACITY_CC', 'CURB_WEIGHT_KG',
       'NO_OF_OWNERS', 'VEHICLE_TYPE'])

We then fill up our dataframe with the information that we have scraped and store it in a .csv file (car_data.csv)

In [38]:
filename = 'car_data_updated'
i = 0 # Indexing rows in the DF

for listingurl in listing_urls:
    response = requests.get(listingurl) 
    listing_url = BeautifulSoup(response.text, 'lxml')
    
    # Retrieval functions to pull data from the Individual Listings after they have been parsed
    df.loc[i, 'LISTING_URL'] = listingurl
    try:
        df.loc[i, 'BRAND'] = brand_retrieval(listing_url)
    except:
        df.loc[i, 'BRAND'] = np.nan

    try:
        df.loc[i, 'PRICE'] = price_retrieval(listing_url)
    except:
        df.loc[i, 'PRICE'] = np.nan

    try:
        df.loc[i, 'DEPRE_VALUE_PER_YEAR'] = depreciation_value_per_year_retrieval(listing_url)
    except:
        df.loc[i, 'DEPRE_VALUE_PER_YEAR'] = np.nan

    try:
        df.loc[i, 'REG_DATE'] = registered_date_retrieval(listing_url)
    except:
        df.loc[i, 'REG_DATE'] = np.nan

    try:
        df.loc[i, 'MILEAGE_KM'] = mileage_retrieval(listing_url)
    except:
        df.loc[i, 'MILEAGE_KM'] = np.nan

    try:
        df.loc[i, 'MANUFACTURED_YEAR'] = manufactured_year_retrieval(listing_url)
    except: 
        df.loc[i, 'MANUFACTURED_YEAR'] = np.nan
    
    try:
        df.loc[i, 'ROAD_TAX_PER_YEAR'] = road_tax_retrieval(listing_url)
    except:
        df.loc[i, 'ROAD_TAX_PER_YEAR'] = np.nan
    
    try:
        df.loc[i, 'TRANSMISSION'] = transmission_retrieval(listing_url)
    except:
        df.loc[i, 'TRANSMISSION'] = np.nan

    try:
        df.loc[i, 'DEREG_VALUE_FROM_SCRAPE_DATE'] = dereg_value_retrieval(listing_url)
    except: 
        df.loc[i, 'DEREG_VALUE_FROM_SCRAPE_DATE'] = np.nan
        
    df.loc[i, 'SCRAPE_DATE'] = datetime.now().strftime("%d/%m/%Y")
    
    try:
        df.loc[i, 'OMV'] = omv_retrieval(listing_url)
    except: 
        df.loc[i, 'OMV'] = np.nan

    try:
        df.loc[i, 'ARF'] = arf_retrieval(listing_url)
    except: 
        df.loc[i, 'ARF'] = np.nan
        
    try:
        df.loc[i, 'POWER'] = power_retrieval(listing_url)
    except:
        df.loc[i, 'POWER'] = np.nan
    
    try:
        df.loc[i, 'COE_FROM_SCRAPE_DATE'] = coe_retrieval(listing_url)
    except:
        df.loc[i, 'COE_FROM_SCRAPE_DATE'] = np.nan
        
    try:
        df.loc[i, 'DAYS_OF_COE_LEFT'] = days_of_coe_retrieval(listing_url)
    except:
        df.loc[i, 'DAYS_OF_COE_LEFT'] = np.nan
        
    try:
        df.loc[i, 'ENGINE_CAPACITY_CC'] = engine_capacity_retrieval(listing_url)
    except: 
        df.loc[i, 'ENGINE_CAPACITY_CC'] = np.nan

    try:
        df.loc[i, 'CURB_WEIGHT_KG'] = curb_weight_retrieval(listing_url)
    except:
        df.loc[i, 'CURB_WEIGHT_KG'] = np.nan
        
    try:
        df.loc[i, 'NO_OF_OWNERS'] = number_of_owners_retrieval(listing_url)
    except:
        df.loc[i, 'NO_OF_OWNERS'] = np.nan
        
    try:
        df.loc[i, 'VEHICLE_TYPE'] = type_of_vehicle_retrieval(listing_url)
    except:
        df.loc[i, 'VEHICLE_TYPE'] = np.nan
    
    df.to_csv("{}.csv".format(filename)) 
    i+=1
    time.sleep(1)




In [39]:
df = pd.read_csv('car_data_updated.csv',index_col=0) 
df = df.dropna()
df

,LISTING_URL,BRAND,PRICE,DEPRE_VALUE_PER_YEAR,REG_DATE,MILEAGE_KM,MANUFACTURED_YEAR,ROAD_TAX_PER_YEAR,TRANSMISSION,DEREG_VALUE_FROM_SCRAPE_DATE,SCRAPE_DATE,OMV,ARF,POWER,COE_FROM_SCRAPE_DATE,DAYS_OF_COE_LEFT,ENGINE_CAPACITY_CC,CURB_WEIGHT_KG,NO_OF_OWNERS,VEHICLE_TYPE
0,https://www.sgcarmart.com/used_cars/info.php?I...,Hyundai,59300.0,14010.0,12-Feb-2018,79000.0,2018,738.0,Auto,22806.0,18/04/2024,11583.0,11583.0,125.0,40000.0,1369.0,1591.0,1345.0,2.0,Mid-Sized Sedan
1,https://www.sgcarmart.com/used_cars/info.php?I...,BMW,152888.0,15950.0,10-Oct-2022,600.0,2022,684.0,Auto,94713.0,18/04/2024,30886.0,35241.0,107.0,80501.0,3071.0,1499.0,1340.0,1.0,Hatchback
2,https://www.sgcarmart.com/used_cars/info.php?I...,Mercedes-Benz,213800.0,26010.0,23-Jun-2021,15700.0,2021,1202.0,Auto,74121.0,18/04/2024,44306.0,54029.0,301.0,46790.0,2619.0,1991.0,1590.0,1.0,Sports Car
9,https://www.sgcarmart.com/used_cars/info.php?I...,Audi,147800.0,19590.0,28-Sep-2020,34000.0,2020,624.0,Auto,58735.0,18/04/2024,36487.0,43082.0,147.0,40989.0,2349.0,1395.0,1470.0,2.0,Sports Car
10,https://www.sgcarmart.com/used_cars/info.php?I...,Volvo,94800.0,20150.0,20-Apr-2018,117000.0,2018,1176.0,Auto,34763.0,18/04/2024,25816.0,28143.0,248.0,37605.0,1461.0,1969.0,1683.0,2.0,SUV
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3991,https://www.sgcarmart.com/used_cars/info.php?I...,Lexus,185000.0,21820.0,30-Aug-2021,22589.0,2021,1784.0,Auto,79091.0,18/04/2024,40320.0,48448.0,203.0,62000.0,2676.0,2487.0,1620.0,1.0,Luxury Sedan
3992,https://www.sgcarmart.com/used_cars/info.php?I...,Porsche,126600.0,23940.0,29-Sep-2009,118000.0,2009,4602.0,Auto,22676.0,18/04/2024,92245.0,92245.0,305.0,42885.0,1918.0,3436.0,1455.0,5.0,Sports Car
3994,https://www.sgcarmart.com/used_cars/info.php?I...,Land,184000.0,28330.0,28-Feb-2019,62000.0,2017,1210.0,Auto,81832.0,18/04/2024,66829.0,92293.0,246.0,36667.0,1469.0,1997.0,1983.0,2.0,SUV
3996,https://www.sgcarmart.com/used_cars/info.php?I...,BMW,96400.0,13080.0,23-Sep-2011,144500.0,2011,2335.0,Auto,42987.0,18/04/2024,42864.0,42864.0,201.0,58306.0,2678.0,2497.0,1650.0,3.0,Luxury Sedan
